# Sampled Softmax

For classification and prediction problems a typical criterion function is cross-entropy with softmax. If the number of output classes is high the computation of this criterion and the corresponding gradients could be quite costly. Sampled Softmax is heuristic to speed up training in these cases.

## Basics

The softmax function is used in neural networks if we want to interpret the network output as a probability distribution over a set of classes $C$ with $|C|=N_C$.

Softmax maps an $N_C$-dimensional vector $z$, which has unrestricted values, to an $N_C$ dimensional vector $p$ whith non-negative values that sum up to 1 so that they can be interpreted as probabilities. More precisely:

$$
\begin{align}
p_i &= softmax(z, i)\\
    &= \frac{exp(z_i)}{\sum_{k\in C} exp(z_k)}\\
\end{align}
$$

In what follows we assume that the input $z$ to the softmax is computed from some hidden vector $h$ of dimension $N_h$  in a specific way, namely:
$$ z = W h + b $$
where $W$ is a learnable weight matrix of dimension $(N_c, N_h)$ and is $b$ a learnable bias vector.
We restrict our self to this specific choice of $z$ because it helps in implementing an efficient sampled softmax.

In a typical use-case like for example a recurrent language model, the hidden vector $h$ would be the output of the recurrent layers of dimension $N_h" and $C$ would be the set of words to predict.   

As a training criterion, we use cross-entropy which is a function of the expected (true) class $t\in C$ and the probability predicted for it:
$$cross\_entropy := -log(p_t)$$

## Sampled Softmax from the outside

For the normal softmax the python api provides the function [cross_entropy_with_softmax](https://cntk.ai/pythondocs/cntk.ops.html?highlight=softmax#cntk.ops.cross_entropy_with_softmax). This takes as input the vector $N_C$-dimensional vector $z$. As mentioned for sampled softmax we assume that this z is computed by $ z = W h + b $. In sampled softmax this has to be part of the whole implementation of the criterion.

Below we show the code for `cross_entropy_with_sampled_softmax_and_embedding`. Let’s look at signature first.

One fundamental difference to the corresponding function in the Python api (`cross_entropy_with_softmax`) is that in the Python api function the input corresponds to $z$ and must have the same dimension as the target vector, while in cross_entropy_with_full_softmax the input corresponds to our hidden vector $h$ can have any dimension (hidden_dim).
Actually, hidden_dim will be typically much lower than the dimension of the target vector.

We also have some additional parameter `num_samples, sampling_weights, allow_duplicates` that control the random sampling. 
Another difference to the api function is that we return a tripple (z, cross_entropy_on_samples, error_on_samples).

We will come boack to the details of the implementation below.


In [ ]:
import numpy as np
import os
import cntk as C

# Creates a model subgraph computing cross-entropy with sampled softmax.
def cross_entropy_with_sampled_softmax_and_embedding(
    hidden_vector,            # Node providing hidden input
    target_vector,            # Node providing the expected labels (as sparse vectors)
    vocab_dim,                # Vocabulary size
    hidden_dim,               # Dimension of the hidden vector
    num_samples,              # Number of samples to use for sampled softmax
    sampling_weights,         # Node providing weights to be used for the weighted sampling
    allow_duplicates = False, # Boolean flag to control whether to use sampling with replacemement (allow_duplicates == True) or without replacement.
    ):
    bias = C.Parameter(shape = (vocab_dim, 1), init = C.init_bias_default_or_0)
    weights = C.Parameter(shape = (vocab_dim, hidden_dim), init = C.init_default_or_glorot_uniform)

    sample_selector_sparse = C.random_sample(sampling_weights, num_samples, allow_duplicates) # sparse matrix [num_samples * vocab_size]
    sample_selector = sample_selector_sparse

    inclusion_probs = C.random_sample_inclusion_frequency(sampling_weights, num_samples, allow_duplicates) # dense row [1 * vocab_size]
    log_prior = C.log(inclusion_probs) # dense row [1 * vocab_dim]

    wS = C.times(sample_selector, weights) # [num_samples * hidden_dim]
    zS = C.times_transpose(wS, hidden_vector) + C.times(sample_selector, bias) - C.times_transpose (sample_selector, log_prior)# [num_samples]

    # Getting the weight vector for the true label. Dimension hidden_dim
    wT = C.times(target_vector, weights) # [1 * hidden_dim]
    zT = C.times_transpose(wT, hidden_vector) + C.times(target_vector, bias) - C.times_transpose(target_vector, log_prior) # [1]


    zSReduced = C.reduce_log_sum(zS)
    
    # Compute the cross entropy that is used for training.
    # We don't check whether any of the classes in the random samples conincides with the true label, so it might
    # happen that the true class is counted
    # twice in the normalising demnominator of sampled softmax.
    cross_entropy_on_samples = C.log_add_exp(zT, zSReduced) - zT

    # For applying the model we also output a node providing the input for the full softmax
    z = C.times_transpose(weights, hidden_vector) + bias
    z = C.reshape(z, shape = (vocab_dim))

    zSMax = C.reduce_max(zS)
    error_on_samples = C.less(zT, zSMax)
    return (z, cross_entropy_on_samples, error_on_samples)




To give a better idea of what the inputs and outputs are and how this all differs from the normal softmax we give below a corresponding function using normal softmax:

In [1]:
# Creates model subgraph computing cross-entropy with (full) softmax.
def cross_entropy_with_softmax_and_embedding(
    hidden_vector,  # Node providing hidden input
    target_vector,  # Node providing the expected labels (as sparse vectors)
    vocab_dim,      # Vocabulary size
    hidden_dim      # Dimension of the hidden vector
    ):
    # Setup bias and weights
    bias = C.Parameter(shape = (vocab_dim, 1), init = C.init_bias_default_or_0)
    weights = C.Parameter(shape = (vocab_dim, hidden_dim), init = C.init_default_or_glorot_uniform)

    
    z = C.reshape( C.times_transpose(weights, hidden_vector) + bias, (1,vocab_dim))
    
    # Use cross_entropy_with_softmax
    cross_entropy = C.cross_entropy_with_softmax(z, target_vector)

    zMax = C.reduce_max(z)
    zT = C.times_transpose(z, target_vector)
    error_on_samples = C.less(zT, zMax)

    return (z, cross_entropy, error_on_samples)

As you can see the main differences to the api function `cross_entropy_with_softmax` are:
* We include an embedding.
* We return a tripple (z, cross_entropy, error_on_samples) instead of just returnting the cross entropy.


## Example

To explain how to integrate sampled softmax let us look at a toy example. In this toy example we first transform one-hot input vectors via some random projection into some lower dimensional vector $h$. The modeling task is to reverse this mapping using (sampled) softmax. Well, as already said this is a toy example.

So let's first look how we could solve this with the normal softmax:

In [ ]:
from math import log, exp, sqrt
import timeit

# Creates 'num_vectors' random one-hot vectors of dimension 'num_classes'.
# This creates tuple with a list of one-hot vectors, and liost with the indices they encode.
def get_random_one_hot_data(num_classes, num_vectors):
    indices = np.random.choice(range(num_classes), size=num_vectors, p=data_sampling_weights).reshape((1,num_vectors))
    list_of_vectors = C.one_hot(indices, num_classes)
    return (list_of_vectors, indices.flatten())

# Create a network that:
# * Transforms the input one hot-vectors with a constant random embedding
# * Applies a linear decoding with parameters we want to learn
# * 
def create_model(labels, vocab_dim, hidden_dim, softmax_sample_size):
    embedding_data = np.random.normal(scale = sqrt(1.0/hidden_dim), size=(vocab_dim, hidden_dim)).astype(np.float32)
    input_embedding = C.constant(shape = (vocab_dim, hidden_dim), value = embedding_data)
    latent_vector = C.times(labels, input_embedding)
    # Connect the latent output to (sampled/full) softmax.
    if use_sampled_softmax:
        sampling_weights = np.asarray(softmax_sampling_weights, dtype=np.float32)
        softmax_input, ce, errs = cross_entropy_with_sampled_softmax(latent_vector, labels, vocab_dim, hidden_dim, softmax_sample_size, sampling_weights, use_sparse = use_sparse)
    else:
        softmax_input, ce, errs = cross_entropy_with_full_softmax(latent_vector, labels, vocab_dim, hidden_dim)

    return softmax_input, ce, errs

def train(vocab_dim, hidden_dim):
    labels = C.input_variable(shape=vocab_dim, is_sparse = use_sparse)
    softmax_input, cross_entropy, errs = create_model(labels, vocab_dim, hidden_dim, softmax_sample_size)

    # Setup the trainer
    lr_per_sample = C.learning_rate_schedule(learning_rate, C.UnitType.sample)
    momentum_time_constant = C.momentum_as_time_constant_schedule(2000)
    learner = C.momentum_sgd(softmax_input.parameters, lr_per_sample, momentum_time_constant, True)
    trainer = C.Trainer(softmax_input, cross_entropy, errs, learner)

    # Run training
    minbatch = 0
    average_cross_entropy = compute_average_cross_entropy(softmax_input, test_set_size, vocab_dim)
    print("minbatch = %d average_cross_entropy = %.3f\tperplexity = %.3f"
            % (minbatch, average_cross_entropy, exp(average_cross_entropy)))

    while True:
        minbatch += 1

        # Specify the mapping of input variables in the model to actual minibatch data to be trained with
        label_data, indices = get_random_one_hot_data(vocab_dim, minibatch_size)
        arguments = ({labels : label_data})

        t_start = timeit.default_timer()
        trainer.train_minibatch(arguments)
        t_end = timeit.default_timer()
        samples_per_second = minibatch_size / (t_end - t_start)
        if minbatch % 10 == 0:
            average_cross_entropy = compute_average_cross_entropy(softmax_input, test_set_size, vocab_dim)
            print("minbatch = %d average_cross_entropy = %.3f perplexity = %.3f samples/s = %.1f"
                    % (minbatch, average_cross_entropy, exp(average_cross_entropy), samples_per_second))
            average_cross_entropy = compute_average_cross_entropy(softmax_input, test_set_size, vocab_dim)
            print("minbatch = %d average_cross_entropy = %.3f perplexity = %.3f samples/s = %.1f"
                    % (minbatch, average_cross_entropy, exp(average_cross_entropy), samples_per_second))

def compute_average_cross_entropy(softmax_input, test_set_size, vocab_dim):
    vectors, indices = get_random_one_hot_data(vocab_dim, test_set_size)
    total_cross_entropy = 0.0
    arguments = (vectors)
    z = softmax_input.eval(arguments).reshape(test_set_size, vocab_dim)

    for i in range(len(indices)):
        log_p = log_softmax(z[i], indices[i])
        total_cross_entropy -= log_p

    return total_cross_entropy / len(indices)

# Computes exp(z[index])/np.sum(exp[z]) for a one-dimensional numpy array in an numerically stable way.
def log_softmax(z,    # numpy array
                index # index into the array
            ):
    max_z = np.max(z)
    return z[index] - max_z - log(np.sum(np.exp(z - max_z)))

def zipf(index):
    return 1.0 / (index + 5.0)

def entropy(p):
    return -np.sum(np.log(p)*p)

if __name__=='__main__':
    # train the LM
    np.random.seed(1)

    softmax_sample_size = 10
    learning_rate = 0.01
    minibatch_size = 100
    vocab_dim = 20
    hidden_dim = 10
    use_sampled_softmax = False
    use_sparse = use_sampled_softmax
    test_set_size = 1000


    zipf_sampling_weights = np.asarray([ zipf(i) for i in range(vocab_dim)], dtype=np.float32)
    data_sampling_weights = zipf_sampling_weights/np.sum(zipf_sampling_weights)
    softmax_sampling_weights =  np.power(data_sampling_weights, 0.5)

    from cntk.device import set_default_device, cpu, gpu

    print("entropy"+str(entropy(data_sampling_weights)))
    train(vocab_dim, hidden_dim)
